# About this notebook
- PyTorch efficientnet_b0 + LGB starter code
- efficientnet_b0 is from https://www.kaggle.com/yasufuminakama/petfinder-efficientnet-b0-starter-training
- LGB is from https://www.kaggle.com/yasufuminakama/petfinder-efficientnet-b0-lgb-training

If this notebook is helpful, feel free to upvote :)

![](https://www.petfinder.my/images/cuteness_meter.jpg)

# Data Loading

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
import shutil
from tqdm import tqdm

In [2]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

RAPIDS version 21.08.01 



In [3]:
crop = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

def get_test_file_path(image_id):
    return "/kaggle/working/crop_images/{}.jpg".format(image_id)

crop['file_path'] = crop['Id'].apply(get_test_file_path)

display(crop.head())

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,file_path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,/kaggle/working/crop_images/4128bae22183829d2b...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,/kaggle/working/crop_images/43a2262d7738e3d420...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,/kaggle/working/crop_images/4e429cead1848a2984...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,/kaggle/working/crop_images/80bc3ccafcc51b6630...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,/kaggle/working/crop_images/8f49844c382931444e...


In [4]:
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

def get_test_file_path(image_id):
    return "/kaggle/input/petfinder-pawpularity-score/test/{}.jpg".format(image_id)

test['file_path'] = test['Id'].apply(get_test_file_path)

display(test.head())

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,file_path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,/kaggle/input/petfinder-pawpularity-score/test...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,/kaggle/input/petfinder-pawpularity-score/test...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,/kaggle/input/petfinder-pawpularity-score/test...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,/kaggle/input/petfinder-pawpularity-score/test...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,/kaggle/input/petfinder-pawpularity-score/test...


# Directory settings

In [5]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './'
MODEL_DIR_1 = '/kaggle/input/nfnet-l1cv183lb181/'
MODEL_DIR_2 = '/kaggle/input/nfnet-l2cv1813lb1795/'
MODEL_DIR_3 = '/kaggle/input/swinl224crop/'
MODEL_DIR_4 = '/kaggle/input/swinl384crop/'
MODEL_DIR_5 = '/kaggle/input/vitlsr384crop/'
#MODEL_DIR_6 = '../input/caits384/'

LGB_MODEL_DIR_1 = '/kaggle/input/nfnet-l1cv183lb181/'
LGB_MODEL_DIR_2 = '/kaggle/input/nfnet-l2cv1813lb1795/'
LGB_MODEL_DIR_3 = '/kaggle/input/vitlsr384crop/'
#LGB_MODEL_DIR_4 = '../input/petfinder-efficientnet-b0-lgb-training/'

SVR_MODEL_DIR_1 = '/kaggle/input/nfnet-l1cv183lb181/'
SVR_MODEL_DIR_2 = '/kaggle/input/nfnet-l2cv1813lb1795/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [6]:
sz = 384
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
test_ids = test_df.Id.to_list()
test_dir = "/kaggle/input/petfinder-pawpularity-score/test/"
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')

In [7]:
!python detect.py\
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt\
--class 15 16\
--img 512\
--conf 0.3\
--iou 0.5\
--source $test_dir\
--name inference\
--save-txt --save-conf --exist-ok

Namespace(agnostic_nms=False, augment=False, classes=[15, 16], conf_thres=0.3, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='inference', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/input/petfinder-pawpularity-score/test/', update=False, view_img=False, weights=['/kaggle/input/ultralyticsyolov5aweights/yolov5x.pt'])
Fusing layers... 
image 1/8 /kaggle/input/petfinder-pawpularity-score/test/4128bae22183829d2b5fea10effdb0c3.jpg: 512x512 Done. (0.035s)
image 2/8 /kaggle/input/petfinder-pawpularity-score/test/43a2262d7738e3d420d453815151079e.jpg: 512x512 Done. (0.033s)
image 3/8 /kaggle/input/petfinder-pawpularity-score/test/4e429cead1848a298432a0acad014c9d.jpg: 512x512 Done. (0.030s)
image 4/8 /kaggle/input/petfinder-pawpularity-score/test/80bc3ccafcc51b66303c2c263aa38486.jpg: 512x512 Done. (0.030s)
image 5/8 /kaggle/input/petfinder-pawpularity-score/test/8f49844c382931444e68dffbe20228f4.jpg: 512x512 Done. (0.030s)
image 6/8 /kaggle/input/petfinder-

In [8]:
os.chdir('/kaggle/working')
save_dir = f'/kaggle/working/crop_images/'
os.makedirs(save_dir, exist_ok=True)

In [9]:
for n, image_id in tqdm(enumerate(test_ids)):
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    orig_image = cv2.imread(f'/kaggle/input/petfinder-pawpularity-score/test/{image_id}.jpg')
    orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    height = orig_image.shape[0]
    width = orig_image.shape[1]
    try:
        file_path = f'/kaggle/working/yolov5/runs/detect/inference/labels/{image_id}.txt'
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        for i, d in enumerate(data):
            xmin.append((d[2]-d[4]/2)*width)
            ymin.append((d[3]-d[5]/2)*height)
            xmax.append((d[2]+d[4]/2)*width)
            ymax.append((d[3]+d[5]/2)*height)
        all_xmin = int(min(xmin))
        all_ymin = int(min(ymin))
        all_xmax = int(max(xmax))
        all_ymax = int(max(ymax))
        all_width_half = (all_xmax - all_xmin) // 2
        all_height_half = (all_ymax - all_ymin) // 2
        r = np.maximum(all_width_half, all_height_half)
        all_xc = (all_xmin + all_xmax) // 2
        all_yc = (all_ymin + all_ymax) // 2
        final_xmin = np.maximum(all_xc-r, 0)
        final_ymin = np.maximum(all_yc-r, 0)
        final_xmax = np.minimum(all_xc+r, width)
        final_ymax = np.minimum(all_yc+r, height)
        crop_img = orig_image[final_ymin:final_ymax, final_xmin:final_xmax, :]
        crop_img = cv2.resize(crop_img, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', crop_img)
    except:
        orig_image = cv2.resize(orig_image, (sz, sz)).astype(np.uint8)
        np.save(save_dir + f'{image_id}', orig_image)

8it [00:00, 317.82it/s]


In [10]:
!rm -r /kaggle/working/yolov5

# CFG

In [11]:
# ====================================================
# CFG
# ====================================================
class CFG:
    num_workers=4
    size_1=512
    size_2=384
    size_3=224
    size_4=384
    batch_size=32
    #batch_size_4=8
    model_name_1='eca_nfnet_l1'
    model_name_2='eca_nfnet_l2'
    model_name_3='swin_large_patch4_window7_224_in22k'
    model_name_4='swin_large_patch4_window12_384_in22k'
    model_name_5='vit_large_r50_s32_384'
    #model_name_6='cait_s36_384'
    seed=42
    target_size=1
    target_col='Pawpularity'
    n_fold=5

# Library

In [12]:
# ====================================================
# Library
# ====================================================
import os
import gc
import sys
import math
import time
import pickle
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from torch.autograd import Variable
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

from sklearn.decomposition import PCA

sys.path.append('/kaggle/input/pytorch-image-models/pytorch-image-models-master')
import timm

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Utils

In [13]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
    return score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

#LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

# Dataset

In [14]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image=image)['image']
        return image

In [15]:
class CropDataset(Dataset):
    def __init__(self, df, transform=None, size=None):
        self.df = df
        self.transform = transform
        self.size = size
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.loc[idx, 'Id']
        img = np.load(f'/kaggle/working/crop_images/{image_id}.npy').astype(np.float32)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.size != None:
            img = cv2.resize(img, (self.size, self.size)).astype(np.float32)
        
        if self.transform:
            sample = self.transform(image=img)
            img = sample['image']
        
        img = (img/255.0 - mean) / std
        img = np.transpose(img, (2, 0, 1))
        img = torch.from_numpy(img.astype(np.float32))

        return img

# Transforms

In [16]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data, size):
    
    if data == 'train':
        return A.Compose([
            A.RandomResizedCrop(size, size, scale=(0.85, 1.0)),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            A.Resize(size, size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# MODEL

In [17]:
# ====================================================
# MODEL
# ====================================================
class NfModel(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model_name = model_name
        self.model = timm.create_model(self.model_name, pretrained=pretrained)
        if  "eff" in self.model_name:
            self.n_features = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
        elif "mixer" in self.model_name:
            self.n_features = self.model.head.in_features
            self.model.head = nn.Identity()
        elif "resnest" in self.model_name:
            self.n_features = self.model.fc.in_features
            self.model.fc = nn.Identity()
        elif "nfnet" in self.model_name:
            self.n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        else:
            self.n_features = self.model.head.in_features
            self.model.head = nn.Identity()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.n_features, 1)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        #feature = self.dropout(feature)
        output = self.fc(feature)
        return output

In [18]:
# ====================================================
# MODEL
# ====================================================
class SwinModel(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model_name = model_name
        self.model = timm.create_model(self.model_name, pretrained=pretrained)
        if  "eff" in self.model_name:
            self.n_features = self.model.classifier.in_features
            self.model.classifier = nn.Identity()
        elif "mixer" in self.model_name:
            self.n_features = self.model.head.in_features
            self.model.head = nn.Identity()
        elif "resnest" in self.model_name:
            self.n_features = self.model.fc.in_features
            self.model.fc = nn.Identity()
        elif "nfnet" in self.model_name:
            self.n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Identity()
        else:
            self.n_features = self.model.head.in_features
            self.model.head = nn.Identity()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.n_features, 1)

    def feature(self, image):
        feature = self.model(image)
        return feature
        
    def forward(self, image):
        feature = self.feature(image)
        output = torch.mean(
            torch.stack(
                [self.fc(self.dropout(feature)) for _ in range(5)],
                dim=0,),
            dim=0,
        )
        return output

# Helper functions

In [19]:
# ====================================================
# Helper functions
# ====================================================
def get_features_preds(test_loader, model, device):
    model.eval()
    features = []
    preds = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (images) in tk0:
        images = images.to(device)
        batch_size = images.size(0)
        with torch.no_grad():
            feature = model.feature(images)
            y_preds = model(images)
        features.append(feature.to('cpu').numpy())
        preds.append(y_preds.sigmoid().detach().cpu() * 100.0)
    features = np.concatenate(features)
    probs = np.concatenate(preds)
    return features, probs

In [20]:
IMG_FEATURES_1 = []
MODEL_PREDS_1 = []

test_dataset = TestDataset(test, transform=get_transforms(data='valid', size=CFG.size_1))
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = NfModel(CFG.model_name_1, pretrained=False)
    state = torch.load(MODEL_DIR_1+f'{CFG.model_name_1}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    IMG_FEATURES_1.append(features)
    MODEL_PREDS_1.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
IMG_FEATURES_2 = []
MODEL_PREDS_2 = []

test_dataset = TestDataset(test, transform=get_transforms(data='valid', size=CFG.size_2))
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = NfModel(CFG.model_name_2, pretrained=False)
    state = torch.load(MODEL_DIR_2+f'{CFG.model_name_2}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    IMG_FEATURES_2.append(features)
    MODEL_PREDS_2.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
MODEL_PREDS_3 = []

test_dataset = CropDataset(crop, transform=None, size=CFG.size_3)
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = SwinModel(CFG.model_name_3, pretrained=False)
    state = torch.load(MODEL_DIR_3+f'{CFG.model_name_3}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    MODEL_PREDS_3.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
MODEL_PREDS_4 = []

test_dataset = CropDataset(crop, transform=None)
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = SwinModel(CFG.model_name_4, pretrained=False)
    state = torch.load(MODEL_DIR_4+f'{CFG.model_name_4}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    MODEL_PREDS_4.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
IMG_FEATURES_5 = []
MODEL_PREDS_5 = []

test_dataset = CropDataset(crop, transform=None)
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = SwinModel(CFG.model_name_5, pretrained=False)
    state = torch.load(MODEL_DIR_5+f'{CFG.model_name_5}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    IMG_FEATURES_5.append(features)
    MODEL_PREDS_5.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

IMG_FEATURES_6 = []
MODEL_PREDS_6 = []

test_dataset = TestDataset(test, transform=get_transforms(data='valid', size=CFG.size_4))
test_loader = DataLoader(test_dataset, 
                         batch_size=CFG.batch_size * 2, 
                         shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

for fold in range(CFG.n_fold):
    model = SwinModel(CFG.model_name_6, pretrained=False)
    state = torch.load(MODEL_DIR_6+f'{CFG.model_name_6}_fold{fold}_best.pth', 
                       map_location=torch.device('cpu'))['model']
    model.load_state_dict(state)
    model.to(device)
    features, preds = get_features_preds(test_loader, model, device)
    IMG_FEATURES_6.append(features)
    MODEL_PREDS_6.append(preds)
    del state; gc.collect()
    torch.cuda.empty_cache()

# LGB

In [25]:
features_1 = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'] + [f"img_{i}" for i in np.arange(np.array(IMG_FEATURES_1).shape[2])]

features_2 = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'] + [f"img_{i}" for i in np.arange(np.array(IMG_FEATURES_2).shape[2])]

features_3 = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'] + [f"img_{i}" for i in np.arange(np.array(IMG_FEATURES_5).shape[2])]
'''
features_4 = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'] + [f"img_{i}" for i in np.arange(np.array(IMG_FEATURES_6).shape[2])]
'''

'\nfeatures_4 = [\'Subject Focus\', \'Eyes\', \'Face\', \'Near\', \'Action\', \'Accessory\',\n            \'Group\', \'Collage\', \'Human\', \'Occlusion\', \'Info\', \'Blur\'] + [f"img_{i}" for i in np.arange(np.array(IMG_FEATURES_6).shape[2])]\n'

In [26]:
def inference_single_lightgbm(test, features, model_path, image_features, fold):
    test[[f"img_{i}" for i in np.arange(np.array(image_features).shape[2])]] = image_features[fold]
    with open(model_path, 'rb') as fin:
        clf = pickle.load(fin)
    prediction = clf.predict(test[features], num_iteration=clf.best_iteration)
    return prediction

In [27]:
def inference_single_svr(test, features, model_path, image_features, fold):
    test[[f"img_{i}" for i in np.arange(np.array(image_features).shape[2])]] = image_features[fold]
    with open(model_path, 'rb') as fin:
        clf = pickle.load(fin)
    prediction = clf.predict(test[features])
    return prediction

In [28]:
#model_predictions_1 = np.mean(MODEL_PREDS_1, 0).reshape(-1)
#model_predictions_2 = np.mean(MODEL_PREDS_2, 0).reshape(-1)
model_predictions_3 = np.mean(MODEL_PREDS_3, 0).reshape(-1)
model_predictions_4 = np.mean(MODEL_PREDS_4, 0).reshape(-1)
model_predictions_5 = np.mean(MODEL_PREDS_5, 0).reshape(-1)
#model_predictions_6 = np.mean(MODEL_PREDS_6, 0).reshape(-1)

lgbm_model_paths_1 = [(fold, LGB_MODEL_DIR_1+f'lightgbm_fold{fold}.pkl') for fold in range(5)]
lgbm_predictions_1 = [inference_single_lightgbm(test, features_1, model_path, IMG_FEATURES_1,fold) for fold, model_path in lgbm_model_paths_1]
lgbm_predictions_1 = np.mean(lgbm_predictions_1, 0)

lgbm_model_paths_2 = [(fold, LGB_MODEL_DIR_2+f'lightgbm_fold{fold}.pkl') for fold in range(5)]
lgbm_predictions_2 = [inference_single_lightgbm(test, features_2, model_path, IMG_FEATURES_2, fold) for fold, model_path in lgbm_model_paths_2]
lgbm_predictions_2 = np.mean(lgbm_predictions_2, 0)

In [29]:
lgbm_model_paths_3 = [(fold, LGB_MODEL_DIR_3+f'lightgbm_fold{fold}.pkl') for fold in range(5)]
lgbm_predictions_3 = [inference_single_lightgbm(test, features_3, model_path, IMG_FEATURES_5, fold) for fold, model_path in lgbm_model_paths_3]
lgbm_predictions_3 = np.mean(lgbm_predictions_3, 0)

lgbm_model_paths_4 = [(fold, LGB_MODEL_DIR_4+f'lightgbm_fold{fold}.pkl') for fold in range(5)]
lgbm_predictions_4 = [inference_single_lightgbm(test, features_4, model_path, IMG_FEATURES_6, fold) for fold, model_path in lgbm_model_paths_4]
lgbm_predictions_4 = np.mean(lgbm_predictions_4, 0)

In [30]:
svr_model_paths_1 = [(fold, SVR_MODEL_DIR_1+f'SVR_fold{fold}.pkl') for fold in range(5)]
svr_predictions_1 = [inference_single_svr(test, features_1, model_path, IMG_FEATURES_1,fold) for fold, model_path in svr_model_paths_1]
svr_predictions_1 = np.mean(svr_predictions_1, 0)

In [31]:
svr_model_paths_2 = [(fold, SVR_MODEL_DIR_2+f'SVR_fold{fold}.pkl') for fold in range(5)]
svr_predictions_2 = [inference_single_svr(test, features_2, model_path, IMG_FEATURES_2,fold) for fold, model_path in svr_model_paths_2]
svr_predictions_2 = np.mean(svr_predictions_2, 0)

In [32]:
!rm -r /kaggle/working/crop_images

In [33]:
os.chdir('/kaggle/working')

In [34]:
models = [svr_predictions_1,model_predictions_3,model_predictions_4,svr_predictions_2,model_predictions_5,lgbm_predictions_3]
w = [0.69, 0.275, 0.18, 0.12, 0.075]

In [35]:
md = models[0]
print(md)
for i,k in enumerate(models[1:]):
    md = w[i]*k + (1-w[i])*md
    print(md)

[40.239258 40.02275  40.07218  40.203373 39.763847 39.99018  39.549763
 40.129272]
[40.357697 41.147263 41.260376 40.557682 40.65953  40.47692  39.675007
 40.42906 ]
[40.184036 40.891056 40.92123  40.338203 40.491467 40.35482  39.764908
 40.273476]
[41.59941  42.406445 42.17369  41.8563   41.838547 41.901173 40.88863
 41.762543]
[40.51482  41.47564  41.165993 40.91267  40.84285  40.839157 39.884647
 40.860058]
[40.78390494 41.66661586 41.39038086 41.19104651 41.03265217 41.04038357
 40.22263508 41.1475628 ]


In [36]:
predictions = md

predictions = (model_predictions_1*0.2 +\
               lgbm_predictions_1*0.2 +\
               model_predictions_2*0.2 +\
               svr_predictions_2*0.2 +\
               model_predictions_3*0.2)

In [37]:
print('sub_prediction')
display(predictions)

sub_prediction


array([40.78390494, 41.66661586, 41.39038086, 41.19104651, 41.03265217,
       41.04038357, 40.22263508, 41.1475628 ])

In [38]:
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
display(test.head())

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,67.75
1,43a2262d7738e3d420d453815151079e,59.15
2,4e429cead1848a298432a0acad014c9d,20.02
3,80bc3ccafcc51b66303c2c263aa38486,94.53
4,8f49844c382931444e68dffbe20228f4,89.82


In [39]:
test['Pawpularity'] = predictions
test.to_csv('/kaggle/working/submission.csv', index=False)
test.to_csv('submission.csv', index=False)
display(test[['Id', 'Pawpularity']].head())

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.783905
1,43a2262d7738e3d420d453815151079e,41.666616
2,4e429cead1848a298432a0acad014c9d,41.390381
3,80bc3ccafcc51b66303c2c263aa38486,41.191047
4,8f49844c382931444e68dffbe20228f4,41.032652
